In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
import os
import h5py

import numpy as np
from torch import Tensor
import d3rlpy
from d3rlpy.dataset import MDPDataset

from dogo.pytorch.dynamics_model.dynamics_model import Gauss_Dynamics_Model, Trainer
from dogo.paths import MODELS_BASEDIR, DATASET_BASEDIR

In [4]:
ALGORITHM = "sac"
ENV = "HalfCheetah-v2"
MODEL_TIMESTAMP_DIR = "2022.05.2022-18:13:40"
DATASET_TIMESTAMP_FILE = "2022.05.2022-11:17:04.h5"
DATASET_PATH = os.path.join(DATASET_BASEDIR, ALGORITHM, ENV, MODEL_TIMESTAMP_DIR, DATASET_TIMESTAMP_FILE)

In [5]:
# with h5py.File(DATASET_PATH, "r") as f:
#     # List all groups
#     print("Keys: %s" % f.keys())
#     a_group_key = list(f.keys())[0]

#     # Get the data
#     data = list(f[a_group_key])

In [6]:
data = MDPDataset.load(DATASET_PATH)

In [7]:
data.terminals.shape

(99900,)

In [39]:
X = Tensor(np.hstack((data.observations[:-1], data.actions[:-1])))
X.shape

torch.Size([99899, 23])

In [40]:
Y = Tensor(np.hstack((data.observations[1:,:], data.rewards[:-1][:, None])))
Y.shape

torch.Size([99899, 18])

In [41]:
model = Gauss_Dynamics_Model(
    input_dims=X.shape[1],
    output_dims=Y.shape[1],
    hidden_units=10,
)

In [47]:
trainer = Trainer(
    model=model,
    n_steps=1000,
)

In [48]:
trainer.train(X,Y)